# Custom Components

This is a guide on how to build a simple app and custom component spec
and launch it via two different schedulers.

See the [Quickstart Guide](quickstart.md) for installation and basic usage.

## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [1]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Overwriting my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [2]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Overwriting my_component.py


We can execute our component via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [3]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

torchx 2022-10-27 17:34:16 INFO     loaded configs from /Users/priyaramani/Workspace/repos/torchx/docs/source/.torchxconfig


torchx 2022-10-27 17:34:16 INFO     Log directory not set in scheduler cfg. Creating a temporary log dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option


torchx 2022-10-27 17:34:16 INFO     Log directory is: /var/folders/ck/dsbpg4794mn2nfwc_wr5_fn40000gn/T/torchx_ukf7jjfm


torchx 2022-10-27 17:34:16 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2022-10-27 17:34:17 INFO     Job finished: SUCCEEDED


local_cwd://torchx/hello_world-pmmv1cm65tmpgd


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [4]:
%%writefile Dockerfile.custom

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Overwriting Dockerfile.custom


Once we have the Dockerfile created we can create our docker image.

In [5]:
%%sh
docker build -t my_app:latest -f Dockerfile.custom .

#1 [internal] load build definition from Dockerfile.custom


#1 sha256:61fb24a3f30b30bf1722ffa46c850b6271dce79cc880cf1315f469ed3b274177


#1 transferring dockerfile: 104B 0.0s done


#1 DONE 0.0s


#2 [internal] load .dockerignore


...

#7 exporting layers done


#7 writing image sha256:eb931e46390433da7ea61b14a54a91affb86c7ce64d33f1c3e7a961f56cf98b4 done


#7 naming to docker.io/library/my_app:latest done


#7 DONE 0.0s


Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them


We can then launch it on the local scheduler.

In [6]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

torchx 2022-10-27 17:34:21 INFO     loaded configs from /Users/priyaramani/Workspace/repos/torchx/docs/source/.torchxconfig


torchx 2022-10-27 17:34:21 INFO     Checking for changes in workspace `file:///Users/priyaramani/Workspace/repos/torchx/docs/source`...


torchx 2022-10-27 17:34:21 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2022-10-27 17:34:23 WARNING  failed to pull image my_app:latest, falling back to local: 404 Client Error for http+docker://localhost/v1.41/images/create?tag=latest&fromImage=my_app: Not Found ("pull access denied for my_app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")


torchx 2022-10-27 17:34:24 INFO     Built new image `sha256:9215cebe702de996f350825c46b569d8b0d00260d90c40b18f8e0f7075c3f2d6` based on original image `my_app:latest` and changes in workspace `file:///Users/priyaramani/Workspace/repos/torchx/docs/source` for role[0]=greeter.


torchx 2022-10-27 17:34:25 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2022-10-27 17:34:26 INFO     Job finished: SUCCEEDED


local_docker://torchx/hello_world-lt9qdxd3xbhgwd


If you have a Kubernetes cluster you can use the [Kubernetes scheduler](schedulers/kubernetes.rst) to launch
this on the cluster instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```

## Builtins

TorchX also provides a number of builtin components with premade images. You can discover
them via:

In [7]:
%%sh
torchx builtins

Found 10 builtin components:


  1. metrics.tensorboard


  2. serve.torchserve


  3. utils.binary


  4. utils.booth


...

  6. utils.echo


  7. utils.python


  8. utils.sh


  9. utils.touch


 10. dist.ddp


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [8]:
%%sh
torchx run utils.echo --msg "Hello :)"

torchx 2022-10-27 17:34:28 INFO     loaded configs from /Users/priyaramani/Workspace/repos/torchx/docs/source/.torchxconfig


torchx 2022-10-27 17:34:29 INFO     Checking for changes in workspace `file:///Users/priyaramani/Workspace/repos/torchx/docs/source`...


torchx 2022-10-27 17:34:29 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2022-10-27 17:34:31 INFO     Built new image `sha256:9215cebe702de996f350825c46b569d8b0d00260d90c40b18f8e0f7075c3f2d6` based on original image `ghcr.io/pytorch/torchx:0.3.0` and changes in workspace `file:///Users/priyaramani/Workspace/repos/torchx/docs/source` for role[0]=echo.


torchx 2022-10-27 17:34:31 INFO     Waiting for the app to finish...


torchx 2022-10-27 17:34:31 INFO     Job finished: SUCCEEDED


echo/0 Hello :)


local_docker://torchx/echo-g4tmlzd03ljjhd
